# Taller #13: Gestos con Cámara Web: Control Visual con MediaPipe
#### Desarrollado por: David Santiago Cruz Hernández

In [1]:
import tkinter as tk
import threading
import cv2
import mediapipe as mp

# pip install -r requirements.txt

### Lista de comandos permitidos

In [2]:
GESTOS_VALIDOS = {'mano_cerrada', 'un_dedo', 'dos_dedos', 'tres_dedos', 'cuatro_dedos', 'mano_abierta'}

### Estados actuales

In [3]:
estado_gesto = None

### Interfaz gráfica con Tkinter

In [4]:
class AppVisual(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Control por Gestos")
        self.geometry("500x400")
        self.color_actual = "white"
        self.pos_x = 200
        self.label = tk.Label(self, text="Esperando comando...", bg=self.color_actual, width=30, height=10, font=("Arial", 20))
        self.label.place(relx=0.5, rely=0.5, anchor="center")


    def actualizar_interfaz(self):
        global estado_gesto
        if estado_gesto == "mano_cerrada":
            self.label.config(text="Mano cerrada detectada", bg="#fbfff4")
        elif estado_gesto == "un_dedo":
            self.label.config(text="Un dedo detectado", bg="#f0ffda")
        elif estado_gesto == "dos_dedos":
            self.label.config(text="Dos dedos detectados", bg="#dafca9")
        elif estado_gesto == "tres_dedos":
            self.label.config(text="Tres dedos detectados", bg="#c7fe77")
        elif estado_gesto == "cuatro_dedos":
            self.label.config(text="Cuatro dedos detectados", bg="#b0ff3d")
        elif estado_gesto == "mano_abierta":
            self.label.config(text="Mano abierta detectada", bg="#97ff00")
        self.after(100, self.actualizar_interfaz)

### Función de detección de gestos

In [5]:
def detectar_gesto_mano(hand_landmarks, mp_hands):
    # Obtiene las posiciones de las puntas y nudillos de los dedos
    dedos = {
        "PULGAR": (mp_hands.HandLandmark.THUMB_TIP, mp_hands.HandLandmark.THUMB_IP),
        "INDICE": (mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.INDEX_FINGER_PIP),
        "MEDIO": (mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP),
        "ANULAR": (mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.RING_FINGER_PIP),
        "MENIQUE": (mp_hands.HandLandmark.PINKY_TIP, mp_hands.HandLandmark.PINKY_PIP),
    }
    extendidos = {}
    for nombre, (tip, pip) in dedos.items():
        extendidos[nombre] = hand_landmarks.landmark[tip].y < hand_landmarks.landmark[pip].y

    # Lógica para los gestos
    if not any(extendidos.values()):
        return "mano_cerrada"
    elif extendidos["INDICE"] and not extendidos["MENIQUE"] and not extendidos["PULGAR"] and not extendidos["ANULAR"] and not extendidos["MEDIO"]:
        return "un_dedo"
    elif extendidos["PULGAR"] and extendidos["MENIQUE"] and not extendidos["INDICE"] and not extendidos["ANULAR"] and not extendidos["MEDIO"]:
        return "dos_dedos"
    elif extendidos["PULGAR"] and extendidos["INDICE"] and extendidos["MENIQUE"] and not extendidos["MEDIO"] and not extendidos["ANULAR"]:
        return "tres_dedos"
    elif extendidos["INDICE"] and extendidos["MEDIO"] and extendidos["PULGAR"] and extendidos["MENIQUE"] and not extendidos["ANULAR"]:
        return "cuatro_dedos"
    elif all(extendidos.values()):
        return "mano_abierta"
    return "otro"

In [6]:
def detectar_gestos():
    global estado_gesto
    mp_hands = mp.solutions.hands
    mp_dibujo = mp.solutions.drawing_utils
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Dibuja los puntos y líneas sobre la mano
                mp_dibujo.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )
                
                # Detecta el gesto de la mano
                estado_gesto = detectar_gesto_mano(hand_landmarks, mp_hands)
        else:
            estado_gesto = None

        cv2.imshow('Gestos de Mano', frame)
        if cv2.waitKey(1) == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

### Actualización periódica de la interfaz

### Ejecutar e Iniciar hilos

In [ ]:
app = AppVisual()
threading.Thread(target=detectar_gestos, daemon=True).start()
app.after(100, app.actualizar_interfaz)
app.mainloop()